# resnet18 分類器模型訓練

使用pytorch提供之模型進行訓練


*   在"常見CNN分類模型"中展示了如何使用pytorch提供之模型進行預測(RESNET、VGG等)，但在實際應用上卻較難與需求吻合(如產品瑕疵類別)，因此此篇將使用"遷移學習"對模型在訓練，使其可以分類出我們想要之類別





## 下載分類影像


*   在製作訓練檔上需要將圖片放置成以下特定格式

(此篇教學已先分好了，按下執行就完事)

    ```
    |-train <- 這是訓練集資料夾
     |-cls1 <-類別1的圖像資料夾
      |-0000.jpg <-類別1的圖像(圖像名稱可隨意)
     |-cls2
     .
    |-val <-這是測試集資料夾
     |-cls1 <-類別1的圖像資料夾
      |-0001.jpg
     |-cls2
    ```

In [1]:
!git clone https://github.com/tetenlost/demo_training_data.git

Cloning into 'demo_training_data'...
remote: Enumerating objects: 114, done.
remote: Total 114 (delta 0), reused 0 (delta 0), pack-reused 114
Receiving objects: 100% (114/114), 2.17 MiB | 23.68 MiB/s, done.


In [2]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data as data
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

## 刪除95％資料，故意造成資料不平衡

In [3]:
import os
import random
def show_file(a_path):
    for item in os.listdir(a_path):
        # 构建子文件夹/文件的完整路径
        item_path = os.path.join(a_path, item)
        # 判断是否为文件夹
        if os.path.isdir(item_path):
            # 获取文件夹内文件数量
            files = len(os.listdir(item_path))
            # 打印结果
            print("{}内有{}个文件。".format(item, files))
show_file('/content/demo_training_data/train/')
# 設定要刪除的檔案比例
delete_percent = 0.95

# 設定資料夾路徑
folder_path = "/content/demo_training_data/train/cardboard"

# 取得資料夾內所有檔案列表
file_list = os.listdir(folder_path)

# 計算要刪除的檔案數量
num_files_to_delete = int(delete_percent * len(file_list))

# 隨機選取要刪除的檔案
files_to_delete = random.sample(file_list, num_files_to_delete)

# 刪除選取的檔案
for file_name in files_to_delete:
    file_path = os.path.join(folder_path, file_name)
    os.remove(file_path)
show_file('/content/demo_training_data/train/')

glass内有40个文件。
cardboard内有40个文件。
glass内有40个文件。
cardboard内有2个文件。


## 宣告資料讀取函式

* 將資料集打包，並做影像處理，方便後續訓練

In [4]:
def build_data(path,batchsize):
    #宣告訓練集影像處理方式
    train_transform = transforms.Compose([
                        #圖像縮放至224*224
                        transforms.Resize((224,224)),
                        #影像隨機水平翻轉(機率50%)
                        transforms.RandomHorizontalFlip(p=0.5),
                        #影像隨機垂直翻轉(機率50%) 
                        transforms.RandomVerticalFlip(p=0.5),
                        #影像之亮度、色相、飽和度隨機轉換
                        transforms.ColorJitter(hue=0.3),
                        #影像旋轉、平移、縮放
                        transforms.RandomAffine(degrees=20, shear=(30,30), translate=(0.2, 0.2)),
                        #將影像轉換為pytorch格式
                        transforms.ToTensor(),
                        #資料標準化至(-1,1)，為了更好的訓練
                        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5]),
                        ])
    #宣告測試集影像處理方式
    val_transform = transforms.Compose([
                        #圖像縮放至224*224
                        transforms.Resize((224,224)),
                        #將影像轉換為pytorch格式
                        transforms.ToTensor(),
                        #資料標準化至(-1,1)，為了更好的訓練
                        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5]),
                        ])
    #使用ImageFolder將圖片與類別作匹配，並添加影像處理
    train_datasets = datasets.ImageFolder(os.path.join(path, 'train'), train_transform)
    val_datasets = datasets.ImageFolder(os.path.join(path, 'val'), val_transform)
    #製作Dataloader，將訓練/測試拆分為batch(如batch_size=16，就是將訓練/測試每16張做為一個batch)
    train_dataloaders = torch.utils.data.DataLoader(train_datasets, batch_size=batchsize,shuffle=True, num_workers=4)
    val_dataloaders = torch.utils.data.DataLoader(val_datasets, batch_size=batchsize,shuffle=True, num_workers=4)
    dataset_sizes={
            'train':len(train_datasets),
            'val':len(val_datasets)
            }
    return train_dataloaders,val_dataloaders,dataset_sizes

## 宣告訓練程式

In [5]:
def train_model(model, train_d, val_d, dataset_sizes, criterion, optimizer, scheduler,device, num_epochs=25):
    #計時
    since = time.time() 
    #將訓練/測試Dataloader製作成字典
    dataloaders={
            'train':train_d,
            'val':val_d
          }
    best_acc = 0.0
    #執行迴圈，總次數為訓練次數
    for epoch in range(num_epochs):
        #顯示幕前的訓練次數
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        #執行迴圈，第一次先執行訓練(train)，在執行測式(val)
        for phase in ['train', 'val']:
            #如果是執行訓練，開啟訓練模式
            if phase == 'train':
                model.train()
            #如果不是，開啟測試模式  
            else:
                model.eval() 
            #初始化本次loss值與正確率
            running_loss = 0.0
            running_corrects = 0
            #執行迴圈，將批次資料從 dataloader中拿出來做訓練/測試
            for inputs, labels in dataloaders[phase]:
                #將圖片放入設備中(CPU/GPU)
                inputs = inputs.to(device)
                #將答案放入設備中(CPU/GPU)
                labels = labels.to(device)
                #初始化優化器
                optimizer.zero_grad()
                #設定是否計算梯度(誤差方向)，關係到模型是否會學習
                with torch.set_grad_enabled(phase == 'train'):
                    #圖像放入模型中進行判定
                    outputs = model(inputs)
                    #選出得分最高之選項
                    _, preds = torch.max(outputs, 1)
                    #將輸出與答案比對，計算loss(誤差大小)
                    loss = criterion(outputs, labels)
                    #如果是訓練中，則將梯度(誤差方向)與loss(誤差大小)回傳至模型中，並更新權重
                    if phase == 'train':
                        loss.backward() 
                        optimizer.step() 
                #將此批次訓練的loss(誤差大小)與正確數進行加總
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            #如果為訓練，更新學習率調整器
            if phase == 'train':
                scheduler.step()
            #計算該次訓練的平均正確率&loss
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            #存下最新的model
            torch.save(model,'./last_model.pth')

            #如果此次結果正確率創下歷史新高，則儲存
            if phase == 'val' and epoch_acc >= best_acc:
                best_acc = epoch_acc
                torch.save(model,'./best_model.pth')

    #計算此次訓練浪費多少時間         
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    #顯示最好的那一次結果
    print('Best val Acc: {:4f}'.format(best_acc))







## 宣告主程式(含訓練參數)
* 在宣告損失函數時添加權重，可以減低資料不平衡造成的問題 

In [7]:
def main():
    #宣告batch size
    bs = 16
    #宣告訓練資料存放點
    path = './demo_training_data'
    #宣告訓練次數
    epoch = 10    
    #取得訓練設備(CPU or GPU)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #獲取官方建構神經網路
    model_ft = models.resnet18(pretrained=True)
    ##以下是遷移學習
    #查看倒數第二層餐數量
    num_ftrs = model_ft.fc.in_features
    #將最後一層更改為我們的層(類別數更改)
    model_ft.fc = nn.Linear(num_ftrs, 2)
    ##遷移學習更改部分結束
    #將模型放入訓練設備(CPU or GPU)
    model_ft.to(device)
    #建立圖像資料集
    train_d,val_d, dsl= build_data(path,bs)
    #宣告損失函數(權重部分[1,1]->[20,1])
    criterion = nn.CrossEntropyLoss(weight=torch.Tensor([1, 1]))
    #宣告優化器
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
    #宣告學習率調整器(每隔一段時間將學習率縮小，以習得更好的權重)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1)
    #將上述宣告的東西放進訓練函式中訓練
    train_model(model_ft,train_d,val_d,dsl,criterion, optimizer_ft, exp_lr_scheduler,device, num_epochs=epoch)

main()


Epoch 0/9
----------
train Loss: 0.4468 Acc: 0.8571
val Loss: 0.9781 Acc: 0.5000
Epoch 1/9
----------
train Loss: 0.2254 Acc: 0.9524
val Loss: 1.5750 Acc: 0.5000
Epoch 2/9
----------
train Loss: 0.1839 Acc: 0.9524
val Loss: 2.0283 Acc: 0.5000
Epoch 3/9
----------
train Loss: 0.2094 Acc: 0.9524
val Loss: 2.2917 Acc: 0.5000
Epoch 4/9
----------
train Loss: 0.2331 Acc: 0.9524
val Loss: 2.2953 Acc: 0.5000
Epoch 5/9
----------
train Loss: 0.1934 Acc: 0.9524
val Loss: 2.1458 Acc: 0.5000
Epoch 6/9
----------
train Loss: 0.1527 Acc: 0.9524
val Loss: 1.8911 Acc: 0.5000
Epoch 7/9
----------
train Loss: 0.1626 Acc: 0.9524
val Loss: 1.5826 Acc: 0.5000
Epoch 8/9
----------
train Loss: 0.1564 Acc: 0.9524
val Loss: 1.3364 Acc: 0.5000
Epoch 9/9
----------
train Loss: 0.1350 Acc: 0.9524
val Loss: 0.9915 Acc: 0.5000
Training complete in 2m 20s
Best val Acc: 0.500000


## 刪除資料集


In [ ]:
!rm -r ./demo_training_data